In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import image_interpolation
import datetime
tf.config.list_physical_devices('GPU')

2022-10-03 19:55:11.048784: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-03 19:55:11.314087: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-03 19:55:12.150412: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-03 19:55:12.150490: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
train,test=tf.keras.datasets.cifar10.load_data()
x_train,y_train=train
x_test,y_test=test
x_train, x_test = x_train / 255.0, x_test / 255.0
print("input shape: ",x_train.shape)
print("output shape: ",y_train.shape)
x_train[0].max()

input shape:  (50000, 32, 32, 3)
output shape:  (50000, 1)


1.0

In [3]:
input=tf.keras.Input(shape=(None, None,3))
num_iterations=10
repeated_input = tf.repeat(tf.expand_dims(input,1), repeats=num_iterations, axis=1)
classifier=tf.keras.Sequential([
    tf.keras.layers.RNN(image_interpolation.ImageSectionRNNCell(grid_dim=(8, 8), units=32)),
    tf.keras.layers.Dense(10, activation='softmax')
])
output=classifier(repeated_input)
model=tf.keras.Model(inputs=input, outputs=output)
model.summary()


2022-10-03 19:55:16.304899: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-03 19:55:16.305835: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-03 19:55:16.306023: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-03 19:55:16.306150: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 tf.expand_dims (TFOpLambda)  (None, 1, None, None, 3)  0        
                                                                 
 tf.repeat (TFOpLambda)      (None, 10, None, None, 3  0         
                             )                                   
                                                                 
 sequential_2 (Sequential)   (None, 10)                11541     
                                                                 
Total params: 11,541
Trainable params: 11,541
Non-trainable params: 0
_________________________________________________________________


In [4]:


log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)
with tf.device('/GPU:0'):
    model.compile(optimizer='adam', metrics=[
                  'accuracy'], loss='sparse_categorical_crossentropy')
    model.fit(x_train, y_train, epochs=10, batch_size=8,
              validation_data=(x_test, y_test),  callbacks=[tensorboard_callback])


2022-10-03 19:55:22.788176: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 614400000 exceeds 10% of free system memory.
2022-10-03 19:55:23.370693: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 614400000 exceeds 10% of free system memory.


Epoch 1/10


2022-10-03 19:55:42.528902: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500
2022-10-03 19:55:43.051377: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-10-03 19:55:43.051761: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-10-03 19:55:43.051781: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-10-03 19:55:43.052132: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-10-03 19:55:43.052172: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


2701/6250 [===========>..................] - ETA: 4:49 - loss: 1.9972 - accuracy: 0.2409

KeyboardInterrupt: 

In [ ]:
print("model prediction: ",model.predict(x_test[0:10]))
print("training label: ",y_test[0:10])

1/1 [==============================] - 0s 88ms/step
model prediction:  [[0.03394961 0.01915349 0.07516911 0.32251146 0.06290946 0.16866095
  0.12968698 0.03938099 0.1355088  0.01306912]
 [0.21517615 0.22667103 0.0115899  0.00432792 0.00759436 0.00342728
  0.0043879  0.00546243 0.37806985 0.14329317]
 [0.2290979  0.04390781 0.00596307 0.00599834 0.00315089 0.00778652
  0.00254403 0.00636241 0.67514277 0.02004624]
 [0.36991525 0.19847985 0.03635517 0.02733427 0.01710863 0.01866996
  0.00605635 0.02640414 0.21512432 0.08455203]
 [0.00474655 0.00092936 0.18427198 0.11328635 0.29349875 0.04598173
  0.3396253  0.01518584 0.0017722  0.00070186]
 [0.00490568 0.00120096 0.14859828 0.26508397 0.09510391 0.18653846
  0.2530367  0.03783382 0.00420143 0.00349679]
 [0.08943404 0.35282275 0.04780743 0.08341175 0.01117207 0.12449456
  0.01681429 0.03328997 0.18115282 0.05960035]
 [0.02556164 0.00111893 0.32108098 0.08083635 0.256102   0.0347085
  0.26867032 0.00713906 0.00391301 0.00086928]
 [0.027374